## This is the final python script for loading and testing of data

In [49]:
import serial
import sklearn
import pickle
from sklearn.model_selection import train_test_split
import numpy as np

k = 5
banks_filled = 1


port = serial.Serial("COM4", 9600)

def scale_and_cast(data):
    for i in range(0,data.shape[1]):
        sf = 255/(max(data[:,i]) - min(data[:,i]))
        data[:,i] = (data[:,i] - min(data[:,i])) * sf

    return np.rint(data)

with open('data.pickle', 'rb') as handle:
    data = pickle.load(handle)

X = data["values"]
y = data["labels"]

print(len(X))

x_train, x_test, y_train, y_test = train_test_split(scale_and_cast(X), y, test_size=int(len(X)-(banks_filled*64)-k))

x_first = x_train[:k]
y_first = y_train[:k]

x_to_load = x_train[k:]
y_to_load = y_train[k:]


print("Ready to send first points")

for val, lab in zip(x_first,y_first):
    #wait for signal
    signal = port.readline()    


    val_list = [int(x) for x in val]
    val_list.append(int(lab))
    #send data
    port.write(bytearray(val_list))

print("Ready to send training data")

for val, lab in zip(x_to_load,y_to_load):
    #wait for signal
    signal = port.readline()    


    val_list = [int(x) for x in val]
    val_list.append(int(lab))
    #send data
    port.write(bytearray(val_list))

print("Ready to send test data")

output = []

for val, label in zip(x_test, y_test):
    #wait for signal
    signal = port.readline()


    val_list = [int(x) for x in val]
    #send data
    port.write(bytearray(val_list))

    signal = port.readline()
    output.append(signal)

int_outputs = [x[0] for x in output]

score = 0
for item, label in zip(int_outputs, y_test):
    if  item == label:
        score += 1

print(score/len(y_test))

5000
Ready to send first points
Ready to send training data
Ready to send test data
0.57980125735145
